# Dataset simpel
In this notebook we will try to create batched with slices of data.
This should bethe final dataform which we will need. 
We will create a tensort with x timepoints with y slices and z observations (x,y,z)



In [5]:
%load_ext autoreload
%autoreload 2
from sys import path
import tensorflow as tf
from scipy.io import arff

path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from src.data.make_dataset import get_eeg
datapath = get_eeg(data_dir="../data/raw")
path = "../data/raw"
from src.data.make_dataset import get_eeg_data
data = get_eeg_data(datapath)

2022-06-09 21:15:53.992 | INFO     | src.data.make_dataset:get_eeg:27 - Data is downloaded to /tmp/.keras/datasets/eeg.
2022-06-09 21:15:53.993 | INFO     | src.data.make_dataset:get_eeg_data:34 - /tmp/.keras/datasets/eeg already exists, skipping download


The chunk iterator we use now uses an iterator to get data from the dataset. It will stop when the obervation changes.
We will also padding to create symetric datasets.


In [3]:
from src.data.make_grouped_chunked_eeg_data import EEGChunkIterator
from src.data.make_eeg_data import EEGListDataset

dsNew = EEGListDataset(data=data[0].tolist())
dataloader = EEGChunkIterator(dsNew, batchsize=2, horizon=3)


We know the data contains 14 observations, with a batchsize of 2 and a horizon of 3 we expect a tensort with the shape [2,3,14]

In [15]:
# gets mostly even sized chunks of 'batch size' 

x = next(iter(dataloader))
x.shape

torch.Size([2, 3, 14])

Great success! We get the correct tensor. Not let's check if the slicing is correct

In [16]:
# with the sliding windows the second window of the first batch 
# should be the same as the first window of the second batch
x[0][1]

tensor([4311.2800, 4011.2800, 4265.1300, 4130.2600, 4339.4900, 4583.5900,
        4089.7400, 4616.4100, 4200.0000, 4220.5100, 4195.9000, 4265.6400,
        4611.2800, 4374.3600], dtype=torch.float64)

In [17]:
x[1][0]
# and this checks out!

tensor([4311.2800, 4011.2800, 4265.1300, 4130.2600, 4339.4900, 4583.5900,
        4089.7400, 4616.4100, 4200.0000, 4220.5100, 4195.9000, 4265.6400,
        4611.2800, 4374.3600], dtype=torch.float64)

In [20]:
for i in range(0,12):
    x = next(iter(dataloader))
    print(x.shape)

torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])
torch.Size([2, 3, 14])


All sensors seem to be the right size, so nice.
However, we want to improve on the performance by buffering instead of iteration in the next notebook